<a href="https://colab.research.google.com/github/aayushme/Aayush_AI_Squad_Project1/blob/master/Musical_Instrumets_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***Kaggle Database***

**Calling Required Libraries**

In [ ]:
import os
import pandas as pd

**Importing Kaggle.json**

In [ ]:
from google.colab import files
my_file=files.upload()

Saving kaggle.json to kaggle.json


**Kaggle Cred**


In [ ]:
cred = pd.read_json("kaggle.json",lines=True)
os.environ['KAGGLE_USERNAME'] = cred.iloc[0][0]
os.environ['KAGGLE_KEY'] = cred.iloc[0][1]

**Downloading Database**

In [ ]:
!kaggle datasets download -d aayushme/music-instruments-classification

 42% 9.00M/21.2M [00:00<00:00, 24.3MB/s]
100% 21.2M/21.2M [00:00<00:00, 47.8MB/s]


In [ ]:
!unzip "*.zip"

In [ ]:
os.listdir("instruments_data")

['acoustic_guitar',
 'flute',
 'bass_guitar',
 'piano',
 'violen',
 'harp',
 'saxophone',
 'drums',
 'tabla',
 'gramophone']

#**Data Augmentation**

**Importing Libraries**

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.vision import *
from fastai.metrics import error_rate
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
img_counter=[]
img_counter.append(10)

**Callling Transform Function and creating data class**

In [ ]:
path=""
tfms = get_transforms(do_flip=True)
data = ImageDataBunch.from_folder( Path("instruments_data"), valid_pct=0.2, bs=64, size=224, ds_tfms=get_transforms())
data.classes

In [ ]:
help(get_transforms)

Help on function get_transforms in module fastai.vision.transform:

get_transforms(do_flip:bool=True, flip_vert:bool=False, max_rotate:float=10.0, max_zoom:float=1.1, max_lighting:float=0.2, max_warp:float=0.2, p_affine:float=0.75, p_lighting:float=0.75, xtra_tfms:Union[Collection[fastai.vision.image.Transform], NoneType]=None) -> Collection[fastai.vision.image.Transform]
    Utility func to easily create a list of flip, rotate, `zoom`, warp, lighting transforms.



**Code to make and save the transforms**

In [ ]:
os.makedirs("image_data",exist_ok=True)
def plots_f(rows, cols, width, height, **kwargs):
  total_plots=plt.subplots(rows,cols,figsize=(width,height))[1].flatten()
  for i,ax in enumerate(total_plots):
    pic.apply_tfms(tfms[0], **kwargs).show(ax=ax)
    plt.savefig("image_data/"+str(img_counter[0])+".jpg")
    img_counter[0]+=1

**Zoom Crop all the images of the dataset**

> We are creating four images form each image of dataset and saving them

> We are using zoom cropping property


In [ ]:
os.makedirs("image_data",exist_ok=True)
path = '/content/instruments_data'
for i in data.classes:
  sub = path + '/' + i
  files = os.listdir(sub)
  print(files)
       

['acoustic_guitar (128).jpg', 'acoustic_guitar (315).jpg', 'acoustic_guitar (101).jpg', 'acoustic_guitar (142).jpg', 'acoustic_guitar (233).jpg', 'acoustic_guitar (78).jpg', 'acoustic_guitar (129).jpg', 'acoustic_guitar (3).jpg', 'acoustic_guitar (305).jpg', 'acoustic_guitar (15).jpg', 'acoustic_guitar (298).jpg', 'acoustic_guitar (191).jpg', 'acoustic_guitar (189).jpg', 'acoustic_guitar (167).jpg', 'acoustic_guitar (351).jpg', 'acoustic_guitar (168).jpg', 'acoustic_guitar (230).jpg', 'acoustic_guitar (80).jpg', 'acoustic_guitar (303).jpg', 'acoustic_guitar (45).jpg', 'acoustic_guitar (326).jpg', 'acoustic_guitar (360).jpg', 'acoustic_guitar (268).jpg', 'acoustic_guitar (88).jpg', 'acoustic_guitar (350).jpg', 'acoustic_guitar (324).jpg', 'acoustic_guitar (362).jpg', 'acoustic_guitar (277).jpg', 'acoustic_guitar (193).jpg', 'acoustic_guitar (89).jpg', 'acoustic_guitar (153).jpg', 'acoustic_guitar (225).jpg', 'acoustic_guitar (259).jpg', 'acoustic_guitar (77).jpg', 'acoustic_guitar (373)

In [ ]:
files[4]

'violen (37).jpg'

In [ ]:
for i in enumerate(files):
  tfms = zoom_crop(scale=(0.75,2), do_rand=True)
  


tfms = zoom_crop(scale=(0.75,2), do_rand=True)
  total_plots=plt.subplots(1,4,figsize=(20,10))[1].flatten()
  for i,ax in enumerate(total_plots):
    open_image(files).apply_tfms(tfms[0], size = 244).show(ax=ax)
    plt.savefig("image_data/"+str(img_counter[0])+".jpg")
    img_counter[0]+=1

#**Training Model**

**Calling Resnet50 into learn variable**

In [ ]:
learn = cnn_learner(data, models.resnet50, metrics=accuracy)

**Training the data**

In [ ]:
learn.fit_one_cycle(4)

**Plotting The Learning Rate**

In [ ]:
learn.unfreeze()
learn.lr_find()
learn.recorder.plot()

**Mounting Drive**

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

**Making Directory in drive and saving path in `my_dir` variable**

In [ ]:
os.makedirs("/content/drive/My Drive/Aayush_Project/digit_recognition",exist_ok=True)
my_dir="/content/drive/My Drive/Aayush_Project/digit_recognition"

**Saving The state with name `Stage-1`**

In [ ]:
learn.save(my_dir+"/Stage-1")

**Export stage-1 to drive**

In [ ]:
learn.export(my_dir+"/export.pkl")